In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.exponential_smoothing import SimpleExpSmoothing
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load data
data = pd.read_csv('electricity_consumption_data.csv', index_col='Date', parse_dates=['Date'])

# Preprocess data
data['Electricity_Consumption'] = pd.to_numeric(data['Electricity_Consumption'], errors='coerce')
data.dropna(inplace=True)

# EDA
plt.figure(figsize=(10,6))
plt.plot(data['Electricity_Consumption'])
plt.title('Electricity Consumption Over Time')
plt.xlabel('Date')
plt.ylabel('Trillion Watts')
plt.show()

# Decomposition Model
decomposition = seasonal_decompose(data['Electricity_Consumption'], model='additive')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# ETS Model
ets_model = SimpleExpSmoothing(data['Electricity_Consumption'])
ets_model_fit = ets_model.fit()

# ARIMA Model
arima_model = ARIMA(data['Electricity_Consumption'], order=(1,1,1))
arima_model_fit = arima_model.fit(disp=0)

# SARIMA Model
sarima_model = ARIMA(data['Electricity_Consumption'], order=(1,1,1), seasonal_order=(1,1,1,12))
sarima_model_fit = sarima_model.fit(disp=0)

# Model Evaluation
models = [decomposition, ets_model_fit, arima_model_fit, sarima_model_fit]
model_names = ['Decomposition', 'ETS', 'ARIMA', 'SARIMA']
error_metrics = []

for model, name in zip(models, model_names):
    forecast = model.forecast(steps=24)  # 2 years = 24 months
    rmse = np.sqrt(mean_squared_error(data['Electricity_Consumption'], forecast))
    rmspe = np.sqrt(np.mean((data['Electricity_Consumption'] - forecast) ** 2)) / np.mean(data['Electricity_Consumption'])
    mape = np.mean(np.abs((data['Electricity_Consumption'] - forecast) / data['Electricity_Consumption'])) * 100
    error_metrics.append([name, rmse, rmspe, mape])

# Model Selection
error_metrics_df = pd.DataFrame(error_metrics, columns=['Model', 'RMSE', 'RMSPE', 'MAPE'])
best_model = error_metrics_df.loc[error_metrics_df['RMSE'].idxmin()]

# Forecasting
best_model_name = best_model['Model']
if best_model_name == 'Decomposition':
    forecast = decomposition.forecast(steps=24)
elif best_model_name == 'ETS':
    forecast = ets_model_fit.forecast(steps=24)
elif best_model_name == 'ARIMA':
    forecast = arima_model_fit.forecast(steps=24)
elif best_model_name == 'SARIMA':
    forecast = sarima_model_fit.forecast(steps=24)

# Output
print('Selected Model:', best_model_name)
print('Error Metrics:')
print(error_metrics_df)
print('Forecasted Electricity Demand for Next 2 Years:')
print(forecast)